## SFINCS - grpc4bmi example

This notebook shows how the SFINCS BMI server built in this package can be used
with grpc4bmi. It assumes you have built the docker container with

```
cd src
docker build -t sfincs-bmiserver .
```

### Requirements

We use grpc4bmi to communicate with the BMI model inside a container.

In [1]:
# pip installa grpc4bmi

### Case specification

SFINCS runs from within the working directory where the input files are located.
CD to the humber case dir

In [2]:
%cd /home/peter/ewatercycle/sfincs-bmi-server/cases/humber/sfincs_humber_executed
%ls

/home/peter/ewatercycle/sfincs-bmi-server/cases/humber/sfincs_humber_executed
figs/        log.txt     sfincs.bzs  sfincs.ind  sfincs.msk  sfincs.src
gis/         precip.nc   sfincs.dep  sfincs.inp  sfincs.obs  sfincs_his.nc
hydromt.log  sfincs.bnd  sfincs.dis  sfincs.man  sfincs.scs  sfincs_map.nc


### Instantiate the model

This starts the model container and connects the client to it.
We can already get the component name

In [3]:
from grpc4bmi.bmi_client_docker import BmiClientDocker
model = BmiClientDocker(image='sfincs-bmiserver', image_port=50051, work_dir="./")
model.get_component_name()

'Sfincs hydrodynamic model (C)'

### Interact with the model

We can initialize the model and see that it had effect: time variables seem to be set from the input file

In [4]:
model.initialize('sfincs.inp')
model.get_current_time(), model.get_start_time(), model.get_end_time()

(950400.0, 950400.0, 1728000.0)

Updating the model should advance the model time. However, the initial timestep
is very small! The get_current_time value doesn't even seem to render to such
precision. This way it will take a million update steps just to advance the
model 1 time unit.

In [5]:
for i in range(10):
    print(model.get_current_time(), model.get_time_step())
    model.update()
    

950400.0 9.999999974752427e-07
950400.0 9.999999974752427e-07
950400.0 9.999999974752427e-07
950400.0 9.999999974752427e-07
950400.0 9.999999974752427e-07
950400.0 9.999999974752427e-07
950400.0 9.999999974752427e-07
950400.0 9.999999974752427e-07
950400.0 9.999999974752427e-07
950400.0 9.999999974752427e-07


Let's try getting it past the first 10 time units...

In [6]:
t = model.get_current_time()
while t < model.get_start_time() + 10:
    model.update()
    t = model.get_current_time()

print(t)

KeyboardInterrupt: 

In [7]:
print(model.get_current_time(), model.get_time_step())

950400.0 9.999999974752427e-07


After 10 minutes, nothing seems to have happened!

When I enforced the timestep to be 1 instead of `model.get_time_step()` (in the
c++ wrapper), the model actually did advance and the time step value also
changed to a more logical value (order of 1).